In [17]:
import pandas as pd
import numpy as np

In [18]:
data = pd.read_csv("bengaluru_house_prices.csv")
data.head()

,area_type,availability,location,size,society,total_sqft,bath,balcony,price
0,Super built-up Area,19-Dec,Electronic City Phase II,2 BHK,Coomee,1056,2.0,1.0,39.07
1,Plot Area,Ready To Move,Chikka Tirupathi,4 Bedroom,Theanmp,2600,5.0,3.0,120.00
2,Built-up Area,Ready To Move,Uttarahalli,3 BHK,NaN,1440,2.0,3.0,62.00
3,Super built-up Area,Ready To Move,Lingadheeranahalli,3 BHK,Soiewre,1521,3.0,1.0,95.00
4,Super built-up Area,Ready To Move,Kothanur,2 BHK,NaN,1200,2.0,1.0,51.00


In [19]:
print("Shape of data:",data.shape)
data = data.drop(['society','availability'],axis='columns')
print("Shape of data:",data.shape)

Shape of data: (13320, 9)
Shape of data: (13320, 7)


In [20]:
data.isnull().sum()

area_type       0
location        1
size           16
total_sqft      0
bath           73
balcony       609
price           0
dtype: int64

In [21]:
data.balcony = data.balcony.fillna(data.balcony.median())
data.bath = data.bath.fillna(data.bath.median())
data = data.dropna()
data.isnull().sum()

area_type     0
location      0
size          0
total_sqft    0
bath          0
balcony       0
price         0
dtype: int64

In [22]:
def is_float(x):
    try:
        float(x)
    except:
        return False
    return True

def convert_sqft_to_num(x):
    tokens = x.split('-')
    if len(tokens) == 2:
        return (float(tokens[0])+float(tokens[1]))/2
    try:
        return float(x)
    except:
        return None


data['bhk'] = data['size'].apply(lambda x: int(x.split(' ')[0]))

data[~data['total_sqft'].apply(is_float)].head()
data.total_sqft = data.total_sqft.apply(convert_sqft_to_num)
data = data[data.total_sqft.notnull()]

data['price_per_sqft'] = data['price']*100000/data['total_sqft']
data.location = data.location.apply(lambda x: x.strip())
location_stats = data['location'].value_counts(ascending=False)

location_stats_less_than_10 = location_stats[location_stats<=10]
data.location = data.location.apply(lambda x: 'other' if x in location_stats_less_than_10 else x)

data = data[~(data.total_sqft/data.bhk<300)]
print("Shape of data:",data.shape)

Shape of data: (12513, 9)


In [23]:
def remove_pps_outliers(df):
    df_out = pd.DataFrame()
    for key, subdf in df.groupby('location'):
        m = np.mean(subdf.price_per_sqft)
        st = np.std(subdf.price_per_sqft)
        reduced_df = subdf[(subdf.price_per_sqft>(m-st)) & (subdf.price_per_sqft<=(m+st))]
        df_out = pd.concat([df_out,reduced_df],ignore_index=True)
    return df_out
data = remove_pps_outliers(data)
print("Shape of data:",data.shape)

Shape of data: (10288, 9)


In [24]:
def remove_bhk_outliers(df):
    exclude_indices = np.array([])
    for location, location_df in df.groupby('location'):
        bhk_stats = {}
        for bhk, bhk_df in location_df.groupby('bhk'):
            bhk_stats[bhk] = {
                'mean': np.mean(bhk_df.price_per_sqft),
                'std': np.std(bhk_df.price_per_sqft),
                'count': bhk_df.shape[0]
            }
        for bhk, bhk_df in location_df.groupby('bhk'):
            stats = bhk_stats.get(bhk-1)
            if stats and stats['count']>5:
                exclude_indices = np.append(exclude_indices, bhk_df[bhk_df.price_per_sqft<(stats['mean'])].index.values)
    return df.drop(exclude_indices,axis='index')
data = remove_bhk_outliers(data)
print("Shape of data:",data.shape)

Shape of data: (7342, 9)


In [25]:
data = data[data.bath<data.bhk+2]
print("Shape of data:",data.shape)

Shape of data: (7264, 9)


In [26]:
data = data.drop(['size','price_per_sqft'],axis='columns')
data = data[data.bhk<=5]
data = data[data.bath<=5]
location_stats_less_than_20 = location_stats[location_stats<20]
data.location = data.location.apply(lambda x: 'other' if x in location_stats_less_than_20 else x)
print("Shape of data:",data.shape)
data.head(3)

Shape of data: (7185, 7)


,area_type,location,total_sqft,bath,balcony,price,bhk
0,Super built-up Area,other,2850.0,4.0,1.0,428.0,4
1,Super built-up Area,other,1630.0,3.0,2.0,194.0,3
2,Super built-up Area,other,1875.0,2.0,3.0,235.0,3


In [27]:
from sklearn.preprocessing import LabelEncoder
AreaTypes = LabelEncoder()
Location = LabelEncoder()
data['area_type'] = AreaTypes.fit_transform(data['area_type'])
data['location'] = Location.fit_transform(data['location'])

data.head(3)

,area_type,location,total_sqft,bath,balcony,price,bhk
0,3,148,2850.0,4.0,1.0,428.0,4
1,3,148,1630.0,3.0,2.0,194.0,3
2,3,148,1875.0,2.0,3.0,235.0,3


In [28]:
X = data.drop(['price'],axis='columns')
y = data.price

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)

In [29]:
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.svm import SVR

level0 = list()
level0.append(('lr', LinearRegression()))
level0.append(('dt', DecisionTreeRegressor()))
level0.append(('rf', RandomForestRegressor()))
level0.append(('ridge', Ridge()))
level0.append(('lasso', Lasso()))
level0.append(('elastic', ElasticNet()))
level0.append(('svr', SVR()))

level1 = LinearRegression()

model = StackingRegressor(estimators=level0, final_estimator=level1, cv=5)

model.fit(X_train, y_train)

StackingRegressor(cv=5,
                  estimators=[('lr', LinearRegression()),
                              ('dt', DecisionTreeRegressor()),
                              ('rf', RandomForestRegressor()),
                              ('ridge', Ridge()), ('lasso', Lasso()),
                              ('elastic', ElasticNet()), ('svr', SVR())],
                  final_estimator=LinearRegression())

In [30]:
prediction = model.predict(X_test)

In [31]:
from sklearn.metrics import r2_score
r2_score(y_test, prediction)

0.8195842301296865

In [32]:
model.fit(X,y)

StackingRegressor(cv=5,
                  estimators=[('lr', LinearRegression()),
                              ('dt', DecisionTreeRegressor()),
                              ('rf', RandomForestRegressor()),
                              ('ridge', Ridge()), ('lasso', Lasso()),
                              ('elastic', ElasticNet()), ('svr', SVR())],
                  final_estimator=LinearRegression())

In [33]:
import pickle
with open('FinalHousePriceModel.pickle','wb') as f:
    pickle.dump(model,f)

with open('AreaTypes.pickle','wb') as f:
    pickle.dump(AreaTypes,f)

with open('Location.pickle','wb') as f:
    pickle.dump(Location,f)

In [46]:
X.head(1)

,area_type,location,total_sqft,bath,balcony,bhk
0,3,148,2850.0,4.0,1.0,4


In [60]:
with open('FinalHousePriceModel.pickle', 'rb') as f:
    model = pickle.load(f)

model.predict([[3,148,2850,4,2,4]])

C:\Users\alway\AppData\Roaming\Python\Python39\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
C:\Users\alway\AppData\Roaming\Python\Python39\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  warnings.warn(
C:\Users\alway\AppData\Roaming\Python\Python39\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
C:\Users\alway\AppData\Roaming\Python\Python39\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but Ridge was fitted with feature names
  warnings.warn(
C:\Users\alway\AppData\Roaming\Python\Python39\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but Lasso was fitted with feature names
  warnings.warn(
C:\Users\alw

array([285.60165977])